## TO DO
- Try out that script is working with new dataset (including the clustering part) with the old functions and properties.
- Shape of nucleus: solve bbox 
- Shape of mitochondria: solve bbox 
- Solve saving in excel file problems
- Probable that there are some small particles classified as cell, nucleus and mitochondria -> determine the minimum size to consider them and filter out the rest.
- Calculate number of each organelle per slide
- Combine cells in 3D:
    - install and try https://pypi.org/project/connected-components-3d/
- Calculate volume of each cell: each slide = 200nm
- Eliminate repeated or blurry sections from the analysis (check in VAST) but after calculating the volume.

In [1]:
from Cells import Cells
from MGFeatures import *
import numpy as np
import pandas as pd
from joblib import Parallel, delayed

C:\Users\Gast-User\.conda\envs\MG\lib\site-packages\numba\core\decorators.py:253: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [2]:
def get_area(labels, labels_list): 
    areas = []
    for i in labels_list:
        area = np.count_nonzero(labels == i)
        areas.append(area)
    areas = np.array(areas)
    return areas

In [3]:
folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\cell'

In [8]:
cell0 = Cells(0)
cell0_labels = cell0.label_cells(folder)

AttributeError: 'str' object has no attribute 'astype'

In [13]:
cell = Cells(0)
imageshape = cell.get_imageshape(folder)
cell_image = cell.img_from_tiles(folder)
cell_labels = cell.label_cells(cell_image)
imageshape

(3, 3, 8192, 8192)

In [36]:
folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\cell'
heterochromatin_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\heterochromatin'
gold_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\gold'
lysosomes_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\lysosome'
mitochondria_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\mitochondria'
nucleus_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\nucleus'
ER_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\er'
golgi_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\golgi'


def make_properties(i):
    properties = {}
    
    # label cells
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) #TODO: this step is redundant! Fix in cells
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    #cell_bboxes = cell.get_bbox(cell_labels)
    
    
    # label organelles
    heterochromatin_labels = mask_it(cell_labels, cell.img_from_tiles(heterochromatin_folder))
    lysosomes_labels = mask_it(cell_labels, cell.img_from_tiles(lysosomes_folder))
    mitochondria_labels = mask_it(cell_labels, cell.img_from_tiles(mitochondria_folder))
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    gold_labels = mask_it(cell_labels, cell.img_from_tiles(gold_folder))
    golgi_labels = mask_it(cell_labels, cell.img_from_tiles(golgi_folder))
    ER = cell.img_from_tiles(ER_folder)
    
    
    # calc properties
    labels_list = np.unique(cell_labels)[1:]

    properties['cell_area'] = get_area(cell_labels, labels_list)
    properties['nucleus_area'] = get_area(nucleus_labels, labels_list)
    properties['heterochromatin_area'] = get_area(heterochromatin_labels, labels_list)
    properties['mitochondria_area'] = get_area(mitochondria_labels, labels_list)
    properties['lysosomal_area'] = get_area(lysosomes_labels, labels_list)
    properties['gold_area'] = get_area(gold_labels, labels_list)
    properties['golgi_area'] = get_area(golgi_labels, labels_list)
    properties['cytoplasm_area'] = properties['cell_area'] - properties['nucleus_area']
    
    properties['ER_lengths'] = ER_length(ER, cell_labels)
    properties['sl_num'] = np.array([i]*len(labels_list))
    
    bbox_rects = []
    for region in regionprops(cell_labels):
        bbox_rect = []
        bbox = region.bbox
        bbox_rect = draw_bbox(bbox)
        bbox_rects.append(bbox_rect)
    properties['bbox_rects'] = bbox_rects
    
    bbox_nucleus = []
    for region_n in regionprops(nucleus_labels):
        bbox_nuc = []
        bbox_n = region_n.bbox
        bbox_nuc = draw_bbox(bbox_n)
        bbox_nucleus.append(np.array(bbox_nuc))
    properties['bbox_nucleus'] = bbox_nucleus
    
    #bbox_mitochondria = []
    #for region_m in regionprops(mitochondria_labels):
    #    bbox_mit = []
    #    bbox_m = region_m.bbox
    #    bbox_mit = draw_bbox(bbox_m)
    #    bbox_mitochondria.append(np.array(bbox_mit))
    #properties['bbox_mitochondria'] = bbox_mitochondria
    
    region = regionprops(cell_labels)
    large = []
    small = []
    for r in region:
        min_row, min_col, max_row, max_col = r.bbox
        D = max(max_row - min_row, max_col - min_col)
        large.append(D)
        d = min(max_row - min_row, max_col - min_col)
        small.append(d)
    properties['cell_large_side'] = np.array(large)
    properties['cell_small_side'] = np.array(small)
    
    properties['roundness'] = (4*properties['cell_area'])/(np.pi*properties['cell_large_side'])
    
    return properties

Use next cell to calculate the function make_properties for one section before applying it to the whole volume (FOR THE WHOLE VOLUME OR A LARGE NUMBER OF SECTIONS USE PARALLEL COMPUTING). 

In [24]:
properties() = make_properties(0)
properties

In [35]:
properties_0['bbox_rects'][0], properties_0['bbox_nucleus'][0]

(array([[ 8192, 20758],
        [ 8478, 20758],
        [ 8478, 21050],
        [ 8192, 21050]]),
 array([[ 8192, 20772],
        [ 8438, 20772],
        [ 8438, 21032],
        [ 8192, 21032]]))

In [37]:
nslices = 2
properties = Parallel(n_jobs=nslices, verbose=True)(delayed(make_properties)(i) for i in range(2))

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:  1.7min remaining:    0.0s


KeyboardInterrupt: 

In [33]:
properties[1]

{'cell_area': array([ 74648,  98476, 174628, 185104, 195436, 102692, 765984,     12,
             4,     44,  24850,  46016, 240256,   5572,    960]),
 'nucleus_area': array([ 53832,   6972,  57468, 116532, 107048,  78828, 355944,      0,
             0,      0,   1940,  21072, 136468,      0,      0]),
 'heterochromatin_area': array([ 22864,   4798,  30828,  49672,  31044,  29544, 144060,      0,
             0,      0,   1940,  15695,  38596,      0,      0]),
 'mitochondria_area': array([  580,  2339,  7808,  1304,  1324,     0, 26048,     0,     0,
            0,     0,     0,  6236,     0,     0]),
 'lysosomal_area': array([    0, 23087,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0]),
 'gold_area': array([1380,  736, 2928,  656, 4316,   24, 8060,    0,    0,    0,  700,
          71, 3328,   96,    0]),
 'golgi_area': array([   0,    0, 3240, 2040,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]),
 'cyt

In [34]:
features = pd.DataFrame()
featuresall = pd.DataFrame()
for i in range(2):
    features = pd.DataFrame()
    features = pd.DataFrame.from_dict(properties[i])
    featuresall = pd.concat([featuresall, features], ignore_index=True)
featuresall

,cell_area,nucleus_area,heterochromatin_area,mitochondria_area,lysosomal_area,gold_area,golgi_area,cytoplasm_area,ER_lengths,sl_num,bbox_rects,cell_large_side,cell_small_side,roundness
0,69760,51040,28008,896,0,1244,0,18720,0.000000,0,"[[8192, 20758], [8478, 20758], [8478, 21050], ...",292,286,304.182160
1,99388,13240,9641,1722,23871,712,0,86148,1835.581945,0,"[[10790, 19482], [11272, 19482], [11272, 19812...",482,330,262.540937
2,230004,99260,39664,8712,0,3344,0,130744,4185.030649,0,"[[11232, 21798], [11744, 21798], [11744, 22404...",606,512,483.251136
3,175100,107216,54276,1292,0,992,1424,67884,2092.246817,0,"[[11274, 20782], [11756, 20782], [11756, 21548...",766,482,291.049927
4,196700,101484,26156,0,0,5100,0,95216,0.000000,0,"[[11288, 5842], [11882, 5842], [11882, 6372], ...",594,530,421.626630
5,89384,70148,20648,0,0,116,0,19236,0.000000,0,"[[14442, 11016], [14836, 11016], [14836, 11376...",394,360,288.850872
6,806788,366644,171216,22504,0,10136,0,440144,4550.216503,0,"[[15024, 18706], [16024, 18706], [16024, 20224...",1518,1000,676.702494
7,40,0,0,0,0,0,0,40,0.000000,0,"[[15362, 20578], [15372, 20578], [15372, 20584...",10,6,5.092958
8,360,0,0,0,0,0,0,360,0.000000,0,"[[15448, 18872], [15456, 18872], [15456, 18944...",72,8,6.366198
9,39125,12360,8412,0,0,675,0,26765,0.000000,0,"[[15593, 7918], [15828, 7918], [15828, 8133], ...",235,215,211.980839


In [4]:
# label cells
cell = Cells(1)
imageshape = cell.get_imageshape(folder) #TODO: this step is redundant! Fix in cells
imageshape

(0, 0, 6594, 6594)

In [9]:
cell = Cells()
imageshape = cell.get_imageshape(folder) #TODO: this step is redundant! Fix in cells
imageshape


(0, 0, 6594, 6594)